In [11]:
import pandas as pd
from datetime import datetime
import psycopg2

### Step 1: Schema Creation

In [14]:
#df = pd.read_csv("filtered_bank_reviews.csv")

conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Smith@19",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Create banks table
cur.execute("""
    CREATE TABLE IF NOT EXISTS banks (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255) UNIQUE NOT NULL
    );
""")

# Create reviews table
cur.execute("""
    CREATE TABLE IF NOT EXISTS reviews (
        id SERIAL PRIMARY KEY,
        bank_id INTEGER REFERENCES banks(id),
        review TEXT,
        rating INTEGER,
        date DATE,
        processed_review TEXT,
        sentiment VARCHAR(50),
        sentiment_score FLOAT
    );
""")

conn.commit()

### Step 2: Insert Data

In [17]:
# Load your cleaned CSV file
df = pd.read_csv("filtered_bank_reviews.csv")

# Convert date column to datetime.date (PostgreSQL expects date format)
df['date'] = pd.to_datetime(df['date']).dt.date

# Insert unique banks first
unique_banks = df['bank'].unique()
for bank in unique_banks:
    cur.execute("""
        INSERT INTO banks (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING;
    """, (bank,))
conn.commit()

# Insert reviews with bank_id lookup
for _, row in df.iterrows():
    cur.execute("SELECT id FROM banks WHERE name = %s;", (row['bank'],))
    bank_id = cur.fetchone()[0]

    cur.execute("""
        INSERT INTO reviews (
            bank_id, review, rating, date, processed_review, sentiment, sentiment_score
        ) VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, (
        bank_id,
        row['review'],
        int(row['rating']),
        row['date'],
        row['processed_review'],
        row['sentiment'],
        float(row['sentiment_score'])
    ))

conn.commit()

# Close connections
cur.close()
conn.close()

In [20]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Step 1: Load cleaned review data
df = pd.read_csv("filtered_bank_reviews.csv")

# Convert 'date' column to datetime.date (PostgreSQL expects DATE format)
df['date'] = pd.to_datetime(df['date']).dt.date

# Step 2: Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Smith@19",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Step 3: Insert unique banks into banks table
unique_banks = df['bank'].unique()

for bank in unique_banks:
    cur.execute("""
        INSERT INTO banks (name)
        VALUES (%s)
        ON CONFLICT (name) DO NOTHING;
    """, (bank,))
conn.commit()

# Step 4: Insert reviews with bank_id reference
for _, row in df.iterrows():
    # Get bank_id from banks table
    cur.execute("SELECT id FROM banks WHERE name = %s;", (row['bank'],))
    bank_row = cur.fetchone()

    if bank_row:
        bank_id = bank_row[0]
        
        # Insert review record
        cur.execute("""
            INSERT INTO reviews (
                bank_id, review, rating, date,
                processed_review, sentiment, sentiment_score
            ) VALUES (%s, %s, %s, %s, %s, %s, %s);
        """, (
            bank_id,
            row['review'],
            int(row['rating']),
            row['date'],
            row['processed_review'],
            row['sentiment'],
            float(row['sentiment_score'])
        ))
    else:
        print(f"Bank not found for review: {row['review']}")  # Optional logging

# Step 5: Commit and close
conn.commit()
cur.close()
conn.close()
print("✅ Data successfully inserted into banks and reviews tables.")


✅ Data successfully inserted into banks and reviews tables.


### Step 3: Verify Rows Inserted

In [22]:
import psycopg2

# Connect to your database
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Smith@19",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# 1. Count total banks
cur.execute("SELECT COUNT(*) FROM banks;")
bank_count = cur.fetchone()[0]
print("Total banks:", bank_count)

# 2. Count total reviews
cur.execute("SELECT COUNT(*) FROM reviews;")
review_count = cur.fetchone()[0]
print("Total reviews:", review_count)

# 3. Check a few rows from banks
cur.execute("SELECT * FROM banks LIMIT 5;")
print("\nSample banks:")
for row in cur.fetchall():
    print(row)

# 4. Check a few rows from reviews
cur.execute("SELECT * FROM reviews LIMIT 5;")
print("\nSample reviews:")
for row in cur.fetchall():
    print(row)

# Clean up
cur.close()
conn.close()

Total banks: 3
Total reviews: 13338

Sample banks:
(1, 'CBE')
(2, 'Dashen')
(3, 'BOA')

Sample reviews:
(1, 1, 'the app is proactive and a good connections.', 5, datetime.date(2025, 6, 5), 'app proactive good connection', 'positive', 0.7)
(2, 1, 'I cannot send to cbebirr app. through this app.', 3, datetime.date(2025, 6, 5), 'send cbebirr app app', 'neutral', 0.0)
(3, 1, 'good', 4, datetime.date(2025, 6, 5), 'good', 'positive', 0.7)
(4, 1, 'not functional', 1, datetime.date(2025, 6, 5), 'functional', 'neutral', 0.0)
(5, 1, "everytime you uninstall the app you have to reach them out physically. very oldy! if that's one of security layer, they'd check for fraud attempt via app source directly(source:- app store, play store ... etc) implicitly ! we are in 2025, physical presence for every app install is traditional(traditionally very rare).", 1, datetime.date(2025, 6, 4), 'everytime uninstall app reach physically oldy one security layer check fraud attempt via app source directly source a

### Check reviews per bank

In [34]:
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Smith@19",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

cur.execute("""
    SELECT b.name, COUNT(r.id) AS review_count
    FROM banks b
    JOIN reviews r ON b.id = r.bank_id
    GROUP BY b.name
    ORDER BY review_count DESC;
""")
print("\nReview count per bank:")
for row in cur.fetchall():
    print(row)

cur.close()
conn.close()



Review count per bank:
('CBE', 11040)
('BOA', 1572)
('Dashen', 726)


###  load into DataFrame

In [35]:
conn = psycopg2.connect(
    dbname="bank_reviews",
    user="postgres",
    password="Smith@19",
    host="localhost",
    port="5432"
)

# Run a query and load into DataFrame
df_reviews = pd.read_sql_query("SELECT * FROM reviews LIMIT 10;", conn)
print(df_reviews)

conn.close()

   id  bank_id                                             review  rating  \
0   1        1       the app is proactive and a good connections.       5   
1   2        1    I cannot send to cbebirr app. through this app.       3   
2   3        1                                               good       4   
3   4        1                                     not functional       1   
4   5        1  everytime you uninstall the app you have to re...       1   
5   6        1  አካውንት የምናስገባበት ቦታ ስም ጽፈን ነው ከዚህ በፊት የላክንባቸውን አ...       4   
6   7        1                                               best       5   
7   8        1                                            bezabih       5   
8   9        1                       Best Mobile Banking app ever       5   
9  10        1  it was good app but it have some issues like i...       2   

         date                                   processed_review sentiment  \
0  2025-06-05                      app proactive good connection  positive

C:\Users\smith\AppData\Local\Temp\ipykernel_11664\1141484068.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_reviews = pd.read_sql_query("SELECT * FROM reviews LIMIT 10;", conn)
